In [1]:
import os
import cv2
from tensorflow.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import numpy as np
from PIL import Image
from skimage import data
from skimage.filters import threshold_otsu
import random

In [2]:
img_folder=r'C:\Users\16645\Desktop\HWR\monkbrill2'

def create_dataset(img_folder):
    img_data_array = []
    # training data    
    
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):            
            #read the image
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path, -1)
            
            # convert the gray image to RBG(3 channels), because we gonna to use the Unet to encode and decode it to a mask image
            RGB_image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
            RGB_image = cv2.resize(RGB_image, (28,28))
            RGB_image = RGB_image.reshape(28,28,3)
            RGB_image = np.array(RGB_image)
            img_data_array.append(RGB_image)
            
    return np.array(img_data_array)

def create_mask_dataset(img_folder):      
    class_name = []
    class_color = []
    # instance label and its corresponding colour
    mask_img_data_array = []
    # array of image Mask
    
    for dir1 in os.listdir(img_folder):
        # set a specific color for each classes
        color = np.array([random.randint(0,255),random.randint(0,255),random.randint(0,255)])
        class_name.append(dir1)
        class_color.append(color)

        for file in os.listdir(os.path.join(img_folder, dir1)):
            #read the image
            image_path = os.path.join(img_folder, dir1,  file)
            image = cv2.imread(image_path, -1)
            image = cv2.resize(image, (28,28))
            #srate to create the mask data set
            height, width = image.shape[0:2]
            thresh = 60
            
            # Convert the row image to a mask image, first convert it into a binary image
            # iterate through each pixel
            for row in range(height):
                for col in range(width):
                    # get the grayscale
                    gray = image[row, col]
                    # If the grayscale value is higher than the threshold, it is equal to the maximum value of 255
                    if gray > thresh:
                        image[row, col] = 255
                    # If it is less than the threshold, it is directly changed to 0
                    elif gray < thresh:
                        image[row, col] = 0
            thresh = threshold_otsu(image)
            binary = image > thresh

            # Make 3 channel RGB image same dimensions
            RGB = np.zeros((binary.shape[0],binary.shape[1],3), dtype=np.uint8)

            # Make True pixels red
            RGB[binary]  = [255,255,255]
            # Make False pixels blue
            RGB[~binary] = color
            color = np.array(color)
            mask_img_data_array.append(RGB) 
            
    return np.array(mask_img_data_array), np.array(class_name), np.array(class_color)


# extract the image array and class name
img_data = create_dataset(r'C:\Users\16645\Desktop\HWR\monkbrill2')
mask_data, class_name, class_color = create_mask_dataset(r'C:\Users\16645\Desktop\HWR\monkbrill2')

In [3]:
img_data.shape
mask_data.shape

(5537, 28, 28, 3)

In [4]:
#generate the synthetic dataset
random.randint(0, mask_data.shape[0] - 1)
synthetic_img_data = []
synthetic_mask_data = []

for i in range(5000):
    a = random.randint(0, mask_data.shape[0] - 1)
    b = random.randint(0, mask_data.shape[0] - 1)
    c = random.randint(0, mask_data.shape[0] - 1)
    d = random.randint(0, mask_data.shape[0] - 1)
    e = random.randint(0, mask_data.shape[0] - 1)
    f = random.randint(0, mask_data.shape[0] - 1)
    g = random.randint(0, mask_data.shape[0] - 1)
    synthetic_img = np.concatenate([img_data[a],img_data[b],img_data[c],img_data[d],img_data[e],img_data[f],img_data[g]], axis=1)  # axis=0纵向  axis=1横向
    synthetic_mask = np.concatenate([mask_data[a],mask_data[b],mask_data[c],mask_data[d],mask_data[e],mask_data[f],mask_data[g]], axis=1)  # axis=0纵向  axis=1横向
    synthetic_img_data.append(synthetic_img)
    synthetic_mask_data.append(synthetic_mask)
    
cv2.imshow('img', synthetic_mask_data[0])
cv2.waitKey()



-1

In [5]:
cv2.imshow('img', synthetic_img_data[0])
cv2.waitKey()

-1

In [6]:
synthetic_img_data = np.array(synthetic_img_data)
synthetic_mask_data = np.array(synthetic_mask_data)
synthetic_mask_data.shape

(5000, 28, 196, 3)

In [7]:
# Give every color a category, the background is 0 and the classes is from 1 to 27

label = synthetic_mask_data
labels = []

def flatLabels(label):
    label_seg = np.zeros(label.shape, dtype=np.uint8)
    label_seg[np.all(label == [255,255,255] , axis=-1)] = 0
    for i in range(27):
        label_seg[np.all(label == class_color[i] , axis=-1)] = i+1
    label_seg = label_seg[:, :, 0]
    return label_seg


for i in range(synthetic_mask_data.shape[0]):
    signal_label = flatLabels(synthetic_mask_data[i])
    labels.append(signal_label)

labels = np.array(labels)
labels = np.expand_dims(labels, axis=3)
labels.shape

(5000, 28, 196, 1)

In [8]:
#one hot encoding to_categorical
n_classes = 28
from keras.utils import to_categorical
labels_cat = to_categorical(labels, num_classes=n_classes)
labels_cat.shape

Using TensorFlow backend.


(5000, 28, 196, 28)

In [9]:
#splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(synthetic_img_data, labels_cat, test_size = 0.10)

In [51]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
import keras_metrics

def UNet(num_classes=28):
    # Input layer
    inputs = Input((28, 196, 3))
    
    # Encoder
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    
    # Bridge
    conv4 = Conv2D(64, 3, activation='relu', padding='same')(pool2)
    conv4 = Conv2D(64, 3, activation='relu', padding='same')(conv4)
    
    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(32, 2, activation='relu', padding='same')(up5)
    
    merge5 = concatenate([conv2, up5], axis=3)
    conv5 = Conv2D(32, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(32, 3, activation='relu', padding='same')(conv5)
    
    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = Conv2D(16, 2, activation='relu', padding='same')(up6)
    merge6 = concatenate([conv1, up6], axis=3)
    conv6 = Conv2D(16, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(16, 3, activation='relu', padding='same')(conv6)
    
    
    # Output layer
    output = Conv2D(num_classes, 1, activation='softmax')(conv6)
    
    model = Model(inputs=inputs, outputs=output)
    
    return model


# Create the UNet model
model = UNet(num_classes=28)
model.summary()
model.compile(optimizer=Adam(lr=0.1), loss=categorical_crossentropy, metrics=[keras_metrics.f1_score()])

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 28, 196, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 28, 196, 16)  448         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 28, 196, 16)  2320        conv2d_39[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 14, 98, 16)   0           conv2d_40[0][0]                  
_______________________________________________________________________________________

In [52]:
history = model.fit(X_train, y_train,
                    batch_size = 16,
                    verbose=1,
                    epochs=2,
                    validation_data=(X_test, y_test),
                    shuffle=False)

Epoch 1/2
Instructions for updating:
`inputs` is now automatically inferred
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
282/282 [==============================] - 69s 246ms/step - loss: 15.7958 - f1_score: 0.0000e+00 - val_loss: 15.8163 - val_f1_score: 0.0000e+00
Epoch 2/2
282/282 [==============================] - 74s 263ms/step - loss: 15.7981 - f1_score: 0.0000e+00 - val_loss: 15.8163 - val_f1_score: 0.0000e+00


In [17]:
img = r'C:\Users\16645\Desktop\HWR\1.jpg'
img = cv2.imread(img,-1)
img = np.array(img)
img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
img = cv2.resize(img,(28,196))
cv2.imshow('img', img)
cv2.waitKey()
img = img.reshape(1,28,196,3)
img = np.array(img)


In [44]:
pre = model.predict(img)[0]

In [45]:
def de_one_hot(one_hot):
    de_onehot = []
    for i in range(len(one_hot)):
        for j in range(28):
            if one_hot[i][j] == 1:
                de_onehot.append(j)
    return de_onehot


pre_1 = []
for i in range(28):
    pre_1.append(de_one_hot(pre[i]))